In [11]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
import datetime

# إنشاء قاعدة البيانات والجداول
def setup_database():
    conn = sqlite3.connect('pharmacy.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Medicine (
            medicine_id INTEGER PRIMARY KEY,
            Name TEXT NOT NULL,
            price REAL NOT NULL,
            quantity INTEGER NOT NULL,
            expiry_date TEXT NOT NULL
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Customer (
            customer_id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            contact_details TEXT NOT NULL
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Sales (
            sale_id INTEGER PRIMARY KEY,
            medicine_name TEXT NOT NULL,
            customer_name TEXT NOT NULL,
            quantity_sold INTEGER NOT NULL,
            date_of_sale TEXT NOT NULL
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS ChangeLog (
            log_id INTEGER PRIMARY KEY,
            operation TEXT NOT NULL,
            table_name TEXT NOT NULL,
            record_id INTEGER,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    conn.commit()
    conn.close()

class PharmacyManagementSystem(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Pharmacy Management System")
        self.geometry("900x700")


        self.tab_control = ttk.Notebook(self)
        self.medicine_tab = ttk.Frame(self.tab_control)
        self.customer_tab = ttk.Frame(self.tab_control)
        self.sales_tab = ttk.Frame(self.tab_control)

        self.tab_control.add(self.medicine_tab, text="Medicines")
        self.tab_control.add(self.customer_tab, text="Customers")
        self.tab_control.add(self.sales_tab, text="Sales")
        self.tab_control.pack(expand=1, fill="both")

        self.create_medicine_tab()
        self.create_customer_tab()
        self.create_sales_tab()

    def create_medicine_tab(self):
        # حقول إدخال الأدوية
        self.Name_label = ttk.Label(self.medicine_tab, text="Medicine Name")
        self.Name_label.grid(column=0, row=0, padx=10, pady=10)
        self.Name_entry = ttk.Entry(self.medicine_tab)
        self.Name_entry.grid(column=1, row=0, padx=10, pady=10)
        self.Name_entry.bind('<Return>', lambda event: self.price_entry.focus())

        self.price_label = ttk.Label(self.medicine_tab, text="Price")
        self.price_label.grid(column=0, row=1, padx=10, pady=10)
        self.price_entry = ttk.Entry(self.medicine_tab)
        self.price_entry.grid(column=1, row=1, padx=10, pady=10)
        self.price_entry.bind('<Return>', lambda event: self.quantity_entry.focus())

        self.quantity_label = ttk.Label(self.medicine_tab, text="Quantity")
        self.quantity_label.grid(column=0, row=2, padx=10, pady=10)
        self.quantity_entry = ttk.Entry(self.medicine_tab)
        self.quantity_entry.grid(column=1, row=2, padx=10, pady=10)
        self.quantity_entry.bind('<Return>', lambda event: self.expiry_date_entry.focus())

        self.expiry_date_label = ttk.Label(self.medicine_tab, text="Expiry Date")
        self.expiry_date_label.grid(column=0, row=3, padx=10, pady=10)
        self.expiry_date_entry = ttk.Entry(self.medicine_tab)
        self.expiry_date_entry.grid(column=1, row=3, padx=10, pady=10)
        self.expiry_date_entry.bind('<Return>', lambda event: self.add_medicine())

        self.add_medicine_button = ttk.Button(self.medicine_tab, text="Add Medicine", command=self.add_medicine)
        self.add_medicine_button.grid(column=1, row=4, padx=10, pady=10)

        self.update_medicine_button = ttk.Button(self.medicine_tab, text="Update Medicine", command=self.update_medicine)
        self.update_medicine_button.grid(column=2, row=4, padx=10, pady=10)

        self.delete_medicine_button = ttk.Button(self.medicine_tab, text="Delete Medicine", command=self.delete_medicine)
        self.delete_medicine_button.grid(column=3, row=4, padx=10, pady=10)

        self.medicine_tree = ttk.Treeview(self.medicine_tab, columns=("ID", "Name", "Price", "Quantity", "Expiry Date"), show='headings')
        self.medicine_tree.heading("ID", text="ID")
        self.medicine_tree.heading("Name", text="Name")
        self.medicine_tree.heading("Price", text="Price")
        self.medicine_tree.heading("Quantity", text="Quantity")
        self.medicine_tree.heading("Expiry Date", text="Expiry Date")
        self.medicine_tree.grid(column=0, row=5, columnspan=4, padx=10, pady=10, sticky='nsew')

        self.load_medicine_tree()

    def create_customer_tab(self):
        # حقول إدخال العملاء
        self.name_label = ttk.Label(self.customer_tab, text="Customer Name")
        self.name_label.grid(column=0, row=0, padx=10, pady=10)
        self.name_entry = ttk.Entry(self.customer_tab)
        self.name_entry.grid(column=1, row=0, padx=10, pady=10)
        self.name_entry.bind('<Return>', lambda event: self.contact_details_entry.focus())

        self.contact_details_label = ttk.Label(self.customer_tab, text="Contact Details")
        self.contact_details_label.grid(column=0, row=1, padx=10, pady=10)
        self.contact_details_entry = ttk.Entry(self.customer_tab)
        self.contact_details_entry.grid(column=1, row=1, padx=10, pady=10)
        self.contact_details_entry.bind('<Return>', lambda event: self.add_customer())

        self.add_customer_button = ttk.Button(self.customer_tab, text="Add Customer", command=self.add_customer)
        self.add_customer_button.grid(column=1, row=2, padx=10, pady=10)

        self.update_customer_button = ttk.Button(self.customer_tab, text="Update Customer", command=self.update_customer)
        self.update_customer_button.grid(column=2, row=2, padx=10, pady=10)

        self.delete_customer_button = ttk.Button(self.customer_tab, text="Delete Customer", command=self.delete_customer)
        self.delete_customer_button.grid(column=3, row=2, padx=10, pady=10)

        self.customer_tree = ttk.Treeview(self.customer_tab, columns=("ID", "name", "Contact"), show='headings')
        self.customer_tree.heading("ID", text="ID")
        self.customer_tree.heading("name", text="name")
        self.customer_tree.heading("Contact", text="Contact")
        self.customer_tree.grid(column=0, row=3, columnspan=4, padx=10, pady=10, sticky='nsew')

        self.load_customer_tree()

    def create_sales_tab(self):
        # حقول إدخال المبيعات
        self.medicine_name_label = ttk.Label(self.sales_tab, text="Medicine Name")
        self.medicine_name_label.grid(column=0, row=0, padx=10, pady=10)
        self.medicine_name_entry = ttk.Entry(self.sales_tab)
        self.medicine_name_entry.grid(column=1, row=0, padx=10, pady=10)
        self.medicine_name_entry.bind('<Return>', lambda event: self.customer_name_entry.focus())

        self.customer_name_label = ttk.Label(self.sales_tab, text="Customer Name")
        self.customer_name_label.grid(column=0, row=1, padx=10, pady=10)
        self.customer_name_entry = ttk.Entry(self.sales_tab)
        self.customer_name_entry.grid(column=1, row=1, padx=10, pady=10)
        self.customer_name_entry.bind('<Return>', lambda event: self.quantity_sold_entry.focus())

        self.quantity_sold_label = ttk.Label(self.sales_tab, text="Quantity Sold")
        self.quantity_sold_label.grid(column=0, row=2, padx=10, pady=10)
        self.quantity_sold_entry = ttk.Entry(self.sales_tab)
        self.quantity_sold_entry.grid(column=1, row=2, padx=10, pady=10)
        self.quantity_sold_entry.bind('<Return>', lambda event: self.date_of_sale_entry.focus())

        self.date_of_sale_label = ttk.Label(self.sales_tab, text="Date of Sale")
        self.date_of_sale_label.grid(column=0, row=3, padx=10, pady=10)
        self.date_of_sale_entry = ttk.Entry(self.sales_tab)
        self.date_of_sale_entry.grid(column=1, row=3, padx=10, pady=10)
        self.date_of_sale_entry.bind('<Return>', lambda event: self.record_sale())

        self.record_sale_button = ttk.Button(self.sales_tab, text="Record Sale", command=self.record_sale)
        self.record_sale_button.grid(column=1, row=4, padx=10, pady=10)

        self.update_sale_button = ttk.Button(self.sales_tab, text="Update Sale", command=self.update_sale)
        self.update_sale_button.grid(column=2, row=4, padx=10, pady=10)

        self.delete_sale_button = ttk.Button(self.sales_tab, text="Delete Sale", command=self.delete_sale)
        self.delete_sale_button.grid(column=3, row=4, padx=10, pady=10)

        self.sales_tree = ttk.Treeview(self.sales_tab, columns=("ID", "Medicine", "Customer", "Quantity", "Date"), show='headings')
        self.sales_tree.heading("ID", text="ID")
        self.sales_tree.heading("Medicine", text="Medicine")
        self.sales_tree.heading("Customer", text="Customer")
        self.sales_tree.heading("Quantity", text="Quantity")
        self.sales_tree.heading("Date", text="Date")
        self.sales_tree.grid(column=0, row=5, columnspan=4, padx=10, pady=10, sticky='nsew')

        self.load_sales_tree()

    def load_medicine_tree(self):
        self.medicine_tree.delete(*self.medicine_tree.get_children())
        conn = sqlite3.connect('pharmacy.db')
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM Medicine')
        medicines = cursor.fetchall()
        for medicine in medicines:
            self.medicine_tree.insert('', tk.END, values=medicine)
        conn.close()

    def load_customer_tree(self):
        self.customer_tree.delete(*self.customer_tree.get_children())
        conn = sqlite3.connect('pharmacy.db')
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM Customer')
        customers = cursor.fetchall()
        for customer in customers:
            self.customer_tree.insert('', tk.END, values=customer)
        conn.close()

    def load_sales_tree(self):
        self.sales_tree.delete(*self.sales_tree.get_children())
        conn = sqlite3.connect('pharmacy.db')
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM Sales')
        sales = cursor.fetchall()
        for sale in sales:
            self.sales_tree.insert('', tk.END, values=sale)
        conn.close()

    def add_medicine(self):
        Name = self.Name_entry.get()
        price = float(self.price_entry.get())
        quantity = int(self.quantity_entry.get())
        expiry_date = self.expiry_date_entry.get()
        medicine = Medicine(Name, price, quantity, expiry_date)
        Medicine.add_medicine(medicine)
        self.log_changes("ADD", "Medicine", Name)
        self.save_changes_to_file("ADD", "Medicine", Name)
        self.load_medicine_tree()

    def update_medicine(self):
        selected_medicine = self.medicine_tree.selection()[0]
        medicine_id = self.medicine_tree.item(selected_medicine, 'values')[0]
        Name = self.Name_entry.get()
        price = float(self.price_entry.get())
        quantity = int(self.quantity_entry.get())
        expiry_date = self.expiry_date_entry.get()
        Medicine.update_medicine(medicine_id, Name, price, quantity, expiry_date)
        self.log_changes("UPDATE", "Medicine", medicine_id)
        self.save_changes_to_file("UPDATE", "Medicine", medicine_id)
        self.load_medicine_tree()

    def delete_medicine(self):
        selected_medicine = self.medicine_tree.selection()[0]
        medicine_id = self.medicine_tree.item(selected_medicine, 'values')[0]
        Medicine.delete_medicine(medicine_id)
        self.log_changes("DELETE", "Medicine", medicine_id)
        self.save_changes_to_file("DELETE", "Medicine", medicine_id)
        self.load_medicine_tree()

    def add_customer(self):
        name = self.name_entry.get()
        contact_details = self.contact_details_entry.get()
        customer = Customer(name, contact_details)
        Customer.add_customer(customer)
        self.log_changes("ADD", "Customer", name)
        self.save_changes_to_file("ADD", "Customer", name)
        self.load_customer_tree()

    def update_customer(self):
        selected_customer = self.customer_tree.selection()[0]
        customer_id = self.customer_tree.item(selected_customer, 'values')[0]
        name = self.name_entry.get()
        contact_details = self.contact_details_entry.get()
        Customer.update_customer(customer_id, name, contact_details)
        self.log_changes("UPDATE", "Customer", customer_id)
        self.save_changes_to_file("UPDATE", "Customer", customer_id)
        self.load_customer_tree()

    def delete_customer(self):
        selected_customer = self.customer_tree.selection()[0]
        customer_id = self.customer_tree.item(selected_customer, 'values')[0]
        Customer.delete_customer(customer_id)
        self.log_changes("DELETE", "Customer", customer_id)
        self.save_changes_to_file("DELETE", "Customer", customer_id)
        self.load_customer_tree()

    def record_sale(self):
        medicine_name = self.medicine_name_entry.get()
        customer_name = self.customer_name_entry.get()
        quantity_sold = int(self.quantity_sold_entry.get())
        date_of_sale = self.date_of_sale_entry.get()
        sale = Sale(medicine_name, customer_name, quantity_sold, date_of_sale)
        Sale.record_sale(sale)
        self.log_changes("ADD", "Sales", medicine_name)
        self.save_changes_to_file("ADD", "Sales", medicine_name)
        self.load_sales_tree()
        
    def update_sale(self):
        selected_sale = self.sales_tree.selection()[0]
        sale_id = self.sales_tree.item(selected_sale, 'values')[0]
        medicine_name = self.medicine_name_entry.get()
        customer_name = self.customer_name_entry.get()
        quantity_sold = int(self.quantity_sold_entry.get())
        date_of_sale = self.date_of_sale_entry.get()
        Sale.update_sale(sale_id, medicine_name, customer_name,quantity_sold,date_of_sale)
        self.log_changes("UPDATE", "Sale", sale_id)
        self.save_changes_to_file("UPDATE", "Sale", sale_id)
        self.load_sales_tree()

    def delete_sale(self):
        selected_sale = self.sales_tree.selection()[0]
        sale_id = self.sales_tree.item(selected_sale, 'values')[0]
        Sale.delete_sale(sale_id)
        self.log_changes("DELETE", "Sale", sale_id)
        self.save_changes_to_file("DELETE", "Sale", sale_id)
        self.load_sales_tree()

    def log_changes(self, operation, table_name, record_id):
        conn = sqlite3.connect('pharmacy.db')
        cursor = conn.cursor()
        cursor.execute('''
            INSERT INTO ChangeLog (operation, table_name, record_id)
            VALUES (?, ?, ?)
        ''', (operation, table_name, record_id))
        conn.commit()
        conn.close()

    def save_changes_to_file(self, operation, table_name, record_id):
        with open('pharmacy_changes.txt', 'a') as file:
            file.write(f"{datetime.datetime.now()}: {operation} - {table_name} - {record_id}\n")

class Medicine:
    def __init__(self, Name, price, quantity, expiry_date):
        self.Name = Name
        self.price = price
        self.quantity = quantity
        self.expiry_date = expiry_date

    @staticmethod
    def add_medicine(medicine):
        conn = sqlite3.connect('pharmacy.db')
        cursor = conn.cursor()
        cursor.execute('''
            INSERT INTO Medicine (Name, price, quantity, expiry_date)
            VALUES (?, ?, ?, ?)
        ''', (medicine.Name, medicine.price, medicine.quantity, medicine.expiry_date))
        conn.commit()
        conn.close()

    @staticmethod
    def update_medicine(medicine_id, Name, price, quantity, expiry_date):
        conn = sqlite3.connect('pharmacy.db')
        cursor = conn.cursor()
        cursor.execute('''
            UPDATE Medicine
            SET Name = ?, price = ?, quantity = ?, expiry_date = ?
            WHERE medicine_id = ?
        ''', (Name, price, quantity, expiry_date, medicine_id))
        conn.commit()
        conn.close()

    @staticmethod
    def delete_medicine(medicine_id):
        conn = sqlite3.connect('pharmacy.db')
        cursor = conn.cursor()
        cursor.execute('DELETE FROM Medicine WHERE medicine_id = ?', (medicine_id,))
        conn.commit()
        conn.close()
    

class Customer:
    def __init__(self, name, contact_details):
        self.name = name
        self.contact_details = contact_details

    @staticmethod
    def add_customer(customer):
        conn = sqlite3.connect('pharmacy.db')
        cursor = conn.cursor()
        cursor.execute('''
            INSERT INTO Customer (name, contact_details)
            VALUES (?, ?)
        ''', (customer.name, customer.contact_details))
        conn.commit()
        conn.close()

    @staticmethod
    def update_customer(customer_id, name, contact_details):
        conn = sqlite3.connect('pharmacy.db')
        cursor = conn.cursor()
        cursor.execute('''
            UPDATE Customer
            SET name = ?, contact_details = ?
            WHERE customer_id = ?
        ''', (name, contact_details, customer_id))
        conn.commit()
        conn.close()

    @staticmethod
    def delete_customer(customer_id):
        conn = sqlite3.connect('pharmacy.db')
        cursor = conn.cursor()
        cursor.execute('DELETE FROM Customer WHERE customer_id = ?', (customer_id,))
        conn.commit()
        conn.close()

class Sale:
    def __init__(self, medicine_name, customer_name, quantity_sold, date_of_sale):
        self.medicine_name = medicine_name
        self.customer_name = customer_name
        self.quantity_sold = quantity_sold
        self.date_of_sale = date_of_sale

    @staticmethod
    def record_sale(sale):
        conn = sqlite3.connect('pharmacy.db')
        cursor = conn.cursor()
        cursor.execute('''
            INSERT INTO Sales (medicine_name, customer_name, quantity_sold, date_of_sale)
            VALUES (?, ?, ?, ?)
        ''', (sale.medicine_name, sale.customer_name, sale.quantity_sold, sale.date_of_sale))
        cursor.execute('''
            UPDATE Medicine
            SET quantity = quantity - ?
            WHERE name = ?
        ''', (sale.quantity_sold, sale.medicine_name))
        conn.commit()
        conn.close()
        
    @staticmethod
    def update_sale(sale_id,medicine_name, customer_name, quantity_sold, date_of_sale):
        conn = sqlite3.connect('pharmacy.db')
        cursor = conn.cursor()
        cursor.execute('''
            UPDATE Sales
            SET medicine_name = ?, customer_name = ? , quantity_sold = ? , date_of_sale = ?
            WHERE sale_id = ?
        ''', (medicine_name, customer_name, quantity_sold, date_of_sale, sale_id))
        conn.commit()
        conn.close()

    @staticmethod
    def delete_sale(sale_id):
        conn = sqlite3.connect('pharmacy.db')
        cursor = conn.cursor()
        cursor.execute('DELETE FROM Sales WHERE sale_id = ?', (sale_id,))
        conn.commit()
        conn.close()

# التحقق من اسم المستخدم وكلمة المرور
def login():
    username = username_entry.get()
    password = password_entry.get()
    if username == "eman" and password == "12345":
        login_window.destroy()
        setup_database()
        app = PharmacyManagementSystem()
        app.mainloop()
    else:
        messagebox.showerror("Error", "Invalid username or password")

# إنشاء نافذة تسجيل الدخول
login_window = tk.Tk()
login_window.title("Login")

username_label = ttk.Label(login_window, text="Username")
username_label.grid(column=0, row=0, padx=10, pady=10)
username_entry = ttk.Entry(login_window)
username_entry.grid(column=1, row=0, padx=10, pady=10)
username_entry.bind('<Return>', lambda event: password_entry.focus())

password_label = ttk.Label(login_window, text="Password")
password_label.grid(column=0, row=1, padx=10, pady=10)
password_entry = ttk.Entry(login_window, show="*")
password_entry.grid(column=1, row=1, padx=10, pady=10)
password_entry.bind('<Return>', lambda event: login())

login_button = ttk.Button(login_window, text="Login", command=login)
login_button.grid(column=1, row=2, padx=10, pady=10)

login_window.mainloop()
